# python to gephi using LLM

In [304]:
## connect to gephi server
from gephistreamer import graph
from gephistreamer import streamer

In [305]:
# create a stream 
stream = streamer.Streamer(streamer.GephiWS(hostname="localhost", port=8080, workspace="workspace1"))


# extract data from Web URL

In [306]:
# extract data from web url
import os 

from langchain_core.documents import Document
from helpers.clean_data import clean_text
from langchain_community.document_loaders import WebBaseLoader

In [307]:
# url_input=('https://en.wikipedia.org/wiki/Dana_White')
# url_input=("https://en.wikipedia.org/wiki/Elon_Musk")
# url_input=('https://en.wikipedia.org/wiki/Mark_Zuckerberg')
# url_input=('https://en.wikipedia.org/wiki/Bill_Gates')
# url_input=('https://en.wikipedia.org/wiki/Jeff_Bezos')
#url_input=('https://en.wikipedia.org/wiki/Steve_Jobs')
#url_input=('https://en.wikipedia.org/wiki/Sam_Altman')
#url_input=('https://en.wikipedia.org/wiki/Mukesh_Ambani')
# url_input=('https://en.wikipedia.org/wiki/Jensen_Huang')
#url_input=('https://en.wikipedia.org/wiki/Satoshi_Nakamoto')
#url_input=('https://en.wikipedia.org/wiki/Donald_Trump')
url_input=('https://en.wikipedia.org/wiki/Larry_Ellison')


loader=WebBaseLoader([url_input])
data=loader.load().pop().page_content
data

'\n\n\n\nLarry Ellison - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life and education\n\n\n\n\n\n\n\n\n2\nEarly career and Oracle\n\n\n\n\nToggle Early career and Oracle subsection\n\n\n\n\n\n2.1\n1977–

In [308]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

data=clean_text(data)
documents=[Document(page_content=data)]
# print(documents)
splitter=RecursiveCharacterTextSplitter(chunk_size=3000,chunk_overlap=100)
smaller_doc=splitter.split_documents(documents)
len(smaller_doc)

19

In [309]:
import os 
from groq import Groq

In [310]:
client=Groq( api_key=os.getenv('GROQ_API_KEY')
)

# try nodes and relationships


In [311]:
system=""" You are a network graph maker tasked with extracting terms and their relations from a given context. Your job is to analyze the provided context chunk (delimited by ```) and extract the ontology of terms mentioned in the context. These terms should represent the key concepts based on the context.

**Guidelines for Extraction:**

1. **Identify Key Terms**:
   - While analyzing the context, identify key terms from each sentence or paragraph.
   - Key terms may include:
     - Objects
     - Entities
     - Locations
     - Organizations
     - Persons
     - Conditions
     - Acronyms
     - Documents
     - Services
     - Concepts, etc.
   - Ensure the terms are as **atomistic** and specific as possible.
   - Ensure to give response only in JSON format and no other texts or anything to indicate the format

2. **Identify Relationships**:
   - Terms mentioned in the same sentence or paragraph are typically related.
   - Each term can have one-on-one relationships with other terms.
   - Determine how terms are related to one another.

3. **Define Relationships**:
   - Clearly identify the nature of the relationship between related terms.
   - Use concise and descriptive language to explain the connection.

**Output Format:**
- Provide your output as a list of JSON objects, with a maximum of 5 term pairs.
- Each object should include:

  - `"node_1"`: A concept extracted from the context.
  - `"node_2"`: A related concept extracted from the context.
  - `"edge"`: A concise sentence describing the relationship between `node_1` and `node_2`.

**Example Output:**
[
   {   
       "node_1": "Climate change",
       "node_2": "Greenhouse gases",
       "edge": "Greenhouse gases contribute significantly to climate change."
   },
   {
       
       "node_1": "Tesla, Inc.",
       "node_2": "Elon Musk",
       "edge": "Elon Musk is the founder and CEO of Tesla, Inc."
   }
]"""

In [312]:
from helpers.prompts import graphPrompt
results=[]
for doc in smaller_doc[:100]:
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system
        },
        {
            "role": "user",
            "content": doc.page_content,
        }
    ],
    model='llama-3.1-8b-instant',
    )
    results.append(chat_completion.choices[0].message.content)

In [313]:
print(results)


['[\n    {\n        "node_1": "Larry Ellison",\n        "node_2": "Oracle Corporation",\n        "edge": "Larry Ellison co-founded Oracle Corporation."\n    },\n    {\n        "node_1": "Larry Ellison",\n        "node_2": "University of Illinois Urbana-Champaign",\n        "edge": "Larry Ellison attended the University of Illinois Urbana-Champaign."\n    },\n    {\n        "node_1": "Larry Ellison",\n        "node_2": "Hawaiian Islands",\n        "edge": "Larry Ellison owns 98% of Lanai, the sixth-largest island in the Hawaiian Islands."\n    },\n    {\n        "node_1": "Larry Ellison",\n        "node_2": "David Ellison",\n        "edge": "Larry Ellison is the father of David Ellison."\n    },\n    {\n        "node_1": "Larry Ellison",\n        "node_2": "Oracle Corporation",\n        "edge": "Larry Ellison served as the chief executive officer of Oracle Corporation from 1977 to 2014."\n    }\n]', '[\n  {\n    "node_1": "Larry Ellison",\n    "node_2": "Illinois University",\n    "edge

In [314]:
from gephistreamer import graph
from gephistreamer import streamer
import json 

stream = streamer.Streamer(streamer.GephiWS(hostname="localhost", port=8080, workspace="workspace1"))


In [315]:
for res in results:
    try: 
        json_object=json.loads(res)
        print(res)
    except Exception as e:
        print('buggy JSON object', e)
    for data in json_object:
        
        node_a = graph.Node(data['node_1'],custom_property=1)
        node_b = graph.Node(data['node_2'],custom_property=2)
        stream.add_node(node_a,node_b)
        edge_ab = graph.Edge(node_a,node_b,custom_property=data['edge'])
        stream.add_edge(edge_ab)
    

[
    {
        "node_1": "Larry Ellison",
        "node_2": "Oracle Corporation",
        "edge": "Larry Ellison co-founded Oracle Corporation."
    },
    {
        "node_1": "Larry Ellison",
        "node_2": "University of Illinois Urbana-Champaign",
        "edge": "Larry Ellison attended the University of Illinois Urbana-Champaign."
    },
    {
        "node_1": "Larry Ellison",
        "node_2": "Hawaiian Islands",
        "edge": "Larry Ellison owns 98% of Lanai, the sixth-largest island in the Hawaiian Islands."
    },
    {
        "node_1": "Larry Ellison",
        "node_2": "David Ellison",
        "edge": "Larry Ellison is the father of David Ellison."
    },
    {
        "node_1": "Larry Ellison",
        "node_2": "Oracle Corporation",
        "edge": "Larry Ellison served as the chief executive officer of Oracle Corporation from 1977 to 2014."
    }
]
[
  {
    "node_1": "Larry Ellison",
    "node_2": "Illinois University",
    "edge": "Larry Ellison attended Universi

In [316]:
node_a = graph.Node("A",custom_property=1)
node_b = graph.Node("D")
node_b.property['custom_property']=1
stream.add_node(node_a,node_b)


In [317]:
edge_ab = graph.Edge(node_a,node_b,custom_property="hello")
stream.add_edge(edge_ab)